In [ ]:
## This notebook shows the functionality of the RobustnessTest.


In [ ]:
#import xaiquantificationtoolbox
import torch
import numpy as np
import h5py
from captum.attr import Saliency, IntegratedGradients
from ...xai_quantification_toolbox.measures.robustness_test import *
from ...xai_quantification_toolbox.loaders.model_interface import *
from ...xai_quantification_toolbox.quantifier.base import *
from ....NoiseGrad.src.models import ResNet18

In [ ]:
### Load model, data and attributions.

In [ ]:
# Load trained pytorch model.
model = ResNet18()
model.load_state_dict(torch.load("PATH_TO_LEARNED_PARAMETERS"))
model.eval()
model = ModelInterface()

# Load data, targets and attributions.
x_batch, y_batch = torch.load("PATH_TO_TEST_LOADER")
a_batch_saliency = Saliency(model).attribute(inputs=x_batch, targets=y_batch, abs=True)

In [ ]:
### Option 1. Evaluate the robustness of attributions in one line of code.

In [ ]:
# One-liner to measure robustness of provided attributions.
scores = RobustnessTest(**{
    "similarity_function": lipschitz_constant,
    "perturbation_function": gaussian_blur,
})(model=model, x_batch=x_batch, y_batch=y_batch, a_batch=a_batch_saliency)

In [ ]:
### Option 2. Evaluate the robustness of provided attributions while enjoying more functionality of Quantifier and Graph.

In [ ]:
# Specify the tests.
tests = [RobustnessTest(**{
    "similarity_function": similarity_fn,
    "perturbation_function": gaussian_blur,
}) for similarity_fn in [lipschitz_constant, distance_euclidean, cosine]]

# Load attributions of another explanation method.
a_batch_intgrad = IntegratedGradients(model).attribute(inputs=x_batch, targets=y_batch)

# Init the quantifier object.
quantifier = Quantifier(measures=tests, io_object=h5py.File("PATH_TO_H5PY_FILE"), checkpoints=..)

# Score the tests.
results = [quantifier.score(model=model, x_batch=x_batch, y_batch=y_batch, a_batch=a_batch)
           for a_batch in [a_batch_saliency, a_batch_intgrad]]

# Plot Saliency vs Integrated Gradients.
Plotting(results, show=False, path_to_save="PATH_TO_SAVE_FIGURE")


